# Capstone Project: The Battle of the Neighborhoods week1 
## Nursing home coverage in Singapore  
#### Applied Data Science Capstone by IBM/Coursera

# Table of contents   
- [Introduction: Business Problem (Week 1)](#1-Introduction:-Business-Problem)
- [Data (Week 1)](#2-Data-mining/-collection)
- Methodology (Week 2)
- Results and Discussion (Week 2)
- Recommendation and Conclusion (Week 2)

# 1 Introduction: Business Problem   
Singapore, like many developed countries, is facing the challenge of a rapidly **aging population** and the **increasing need to provide long-term care services for elderly in the community**. Ideally, family members are the first line of support and care for the elderly. However, long term care facilities like nursing home is in demand to care for elderly that required constant medical and/or nursing supervision.   

#### Nursing homes in Singapore    
Run by either the private sector or social service agencies in singapore, nursing homes are essentially long-term residential care homes which provide many services to its residents who are unable to be cared for at home by caregivers or sevice providers including:
- Assistance with activities for daily living such as showering, going to the toilet and eating.
- Skilled nursing care and supervision 24/7.
- Rehabilitation therapies and programmes to stimulate cognitive functions.

That said, it is important to note that *not everyone* can be admitted into a nursing home in Singapore. Whether or not an elderly person can enrol in one depends on his or her eligibility. These include:
- Being physically or mentally disabled due to an illness.
- Facing mobiilty issues including being wheelchair-reliant or bedbound.
- Being unable to be cared for at home by their family or a paid domestic worker and have tried all other care options including daycare and home nursing care (trained nurses to manage specialized nursing care needs in the comfort of your own homes, charges hourly or long-hour package), among others.    

[*Source*](https://www.income.com.sg/blog/nursing-homes-in-singapore)

#### MOH (Ministry of Health) to grow aged care services to meet rising demand    
The Lien report (year 2018) stressed the importance of preventive care, citing how a rapidly greying Singapore could see the number of people aged 65 years and older doubling to a million by 2030.    
In responds to Lien report which found that more seniors are using subsidised home and centre-based care services, MOH said that Singapore has sufficient capacity currently to meet demand for aged care, but it **will continue to grow services to cater to a greying nation**.   
[*source*](https://www.straitstimes.com/singapore/sufficient-capacity-to-meet-demand-in-aged-care-says-moh-in-response-to-lien-foundation)


#### Objective   
If we want to be sustainable as an ageing society and age healthily and happily we need to put in the steps now rather than wait for the demographic explosion to take place. Is nursing home or long-term care service for elderly is able to meet the demand by then?   
The aim of this project is to understand the nursing home coverage in Singapore. Which are has the most nursing home? Is nursing home roughly distributed to all planning area? Are there any particular subzone in any of the planning area is relatively having lower nursing home coverage, based on demographic trait? If yes, any strategic location to recommand for new nursing home facilities? There are the questions that I would like to find the answer in this project.

#### Who would be interested (Target audience)
The outcome would be good information for both general public as well as government agencies. 
MOH, nursing home operator, social service agencies is able to prioritze resources to target neighborhood with low nursing home coverage.

# 2 Data mining/ collection  
In this project, five different datasets will be used to solve the problem. They are listed below:
1. List of planning areas of Singapore. There are total of 55 planning areas. `Population density` for each planning area will be used. In addition, coordinates for each planning area will be compiled via geocoding.
2. Singapore demographics. `Population` for different `age group` from subzone for each planning area will be used.
3. Singapore boundary. `Polygon` for each planning area and subzone from each planning area will be used.
4. List of nursing home. `Coordinates` for each nursing home will be used to visualize their distribution across Singapore. 
5. List of `top popular places` for each planning area to obtain via Foursquare location data. The data will be used to analyze the 55 planning area and divided them into **8 clusters**. I will analyze the density distribution of nursing home per planning area and which cluster are they belongs to.   
In addition, with the coordinates for each nursing home, I will be able to visualize their coverage (by defining the area of coverage = **1km radius** from the nursing home location). Based on the data to find is there any area with low coverage (based on the senior age group population and number of nursing home within the planning area).




In [4]:
# Required libraries are imported in this code cell
import pandas as pd
import geopandas as gpd
import numpy as np

# libraries needed for data collection
import requests
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim
import json

# for map plotting
import folium

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns

# for clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

%matplotlib inline

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## 2.1 Web scraping    
Data collected via web scraping are:
- [Planning areas of Singapore](https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore)
- [Nursing homes of Singapore](https://www.healthhub.sg/directory/nursing-homes)

### 2.1.1 Planning areas of Singapore     
Wiki page shown in url in below code cell is the page that I obtain the list of 55 planning areas of Singapore.
The table also include the population (*2018 latested data*), area size as well as the population density (/km2).   
I'll use the list of planning area to find its coordinates. This information will be used to get the top nearby places, whih is the X-features for neighborhood clustering.

In [3]:
# Define url:
url = 'https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore'

# Establishing the connection on the webpage:
results_text = requests.get(url).text

In [4]:
# Creating beautifulsoup object
soup = BeautifulSoup(results_text, 'lxml')

# table that I want to extract is the table , with class = 'wikitable sortable'
table = soup.find('table', {'class': 'wikitable sortable'})

# find all table row (tr)
table_rows = table.find_all('tr')

In [5]:
# create empty list to store the data row by iterate through the table rows
column_name = []
row_data = []

for row in table_rows[:1]:
    column_name.append([t.text.strip() for t in row.find_all('th')])
for row in table_rows[1:]:
    row_data.append([t.text.strip() for t in row.find_all('td')])

In [6]:
# convert data into dataframe (df_sg)
df_sg = pd.DataFrame(row_data, columns = column_name[0])

# drop unwanted column (keep only the name in English)
df_sg.drop(['Malay', 'Chinese', 'Pinyin', 'Tamil'], axis = 1, inplace = True)
df_sg.columns = ['Name', 'Region', 'Area(km2)', 'Population', 'Density(/km2)']
df_sg.head()

,Name,Region,Area(km2),Population,Density(/km2)
0,Ang Mo Kio,North-East,13.94,"163,950","13,400"
1,Bedok,East,21.69,"279,380","13,000"
2,Bishan,Central,7.62,"88,010","12,000"
3,Boon Lay,West,8.23,30,3.6
4,Bukit Batok,West,11.13,"153,740","14,000"


In [7]:
#alternative approaches to pull table from wiki page
#pd.read_html(url, header = 0)[2]

In [8]:
# datatype for population and density are object (str) to convert them into numeric datatype in the next code cell
df_sg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           55 non-null     object
 1   Region         55 non-null     object
 2   Area(km2)      55 non-null     object
 3   Population     55 non-null     object
 4   Density(/km2)  55 non-null     object
dtypes: object(5)
memory usage: 2.3+ KB


In [9]:
# remove comma for column so that it can convert to numeric datatype
df_sg['Population'] = df_sg['Population'].apply(lambda x:x.replace(',',''))
df_sg['Density(/km2)'] = df_sg['Density(/km2)'].apply(lambda x:x.replace(',',''))

In [10]:
# convert to numeric datatype
col = ['Population', 'Density(/km2)']
df_sg[col] = df_sg[col].apply(pd.to_numeric, errors='coerce', axis =1) #'coerce' => invalid parsing will be set to NaN

In [11]:
# Impute NaN for column: 'Population' & 'Density' columns with Zero => indicating no people living in the area
values = {'Population': 0, 'Density(/km2)': 0}
df_sg.fillna(value=values, inplace = True )

### 2.1.2 Nursing homes of Singapore    
The `url` shown in below code cell is the webpage that proivdes the list of nursing homes in Singapore.
I've used the `requests` and `BeautifulSoup` library to scrape for the name of nursing home and its corresponding location coordinates.   
This data is to used to find out the distribution of the nursing home in Singapore and analyze its coverage per planning area, which is the objective of this project.

In [12]:
# Define url:
url = 'https://www.healthhub.sg/directory/nursing-homes'

# Establishing the connection on the webpage:
results_text = requests.get(url).text

In [13]:
# Creating beautifulsoup object
soup = BeautifulSoup(results_text, 'html.parser')
type(soup)

bs4.BeautifulSoup

All of the information for each nursing home is contained in a `div` tag.   
Using `find_all()` method to extract all the `div` containers that have class attribute of `panel panel-default`

In [14]:
nursinghome_containers = soup.find_all('div', class_ = "panel panel-default")
print(type(nursinghome_containers))
print(f" Total {len(nursinghome_containers)} nursing homes listed in one page")

<class 'bs4.element.ResultSet'>
 Total 16 nursing homes listed in one page


Data to scrape is the name of nursing home and the latitude and longitude.    
Code cell below shows where I identify these info from one nursing home, i.e. the first nursing home in the list

In [15]:
first_home = soup.find('div', class_ = "panel panel-default")
# The name of the home + address is store in `span` tag, and `app_ment` class attribute
first_home.div.a.find('span', class_ = "app_ment").text.strip().split('\n')

['ALL SAINTS HOME (HOUGANG)', ' 5 Poh Huat Road Singapore 546703']

In [16]:
# The latitude and longitude info can be extracted from the url link
# The link is store in `div` tag and 'Rectangle-2' class attribute
link = first_home.find('div', class_ = "Rectangle-2").find('a')['href']
link

'https://www.google.com.sg/maps?q=1.365532,103.883965'

In [17]:
# Extract the latitude and longitude from the link using regular expression (re)
latlng = re.findall('q=(.+)', link)[0].split(',')
latlng

['1.365532', '103.883965']

Define function to convert the latitude and longitude into float and split them into tuple

In [18]:
def convert_latlng2float(latlng_str):
    to_float = list(map(float, latlng_str))
    lat = to_float[0]
    lng = to_float[1]
    return (lat,lng)

Code cell below is to extract all these info from all the nursing homes from the webpage

In [19]:
names = []
address = []
latitude = []
longitude = []

# Extract data from individual nursinghome_containers
for container in nursinghome_containers:
    # if the nursing home has content, then extract
    if container.div.a.find('span', class_ = "app_ment") is not None:
        # Find the nursing home name & its address under class "app_ment"
        home_add = container.div.a.find('span', class_ = "app_ment").text.strip().split('\n')
        names.append(home_add[0])
        address.append(home_add[1])
        
        # Find the latitude & longitude under tag 'div' and class "Rectangle-2"
        # It returns html link
        link = container.find('div',  class_ = "Rectangle-2").find('a')['href']
        # extract the latitude & longitude from the link
        latlng = re.findall('q=(.+)', link)[0].split(',')
        # use the function to convert lat & log into float and split them
        lat,lng = convert_latlng2float(latlng)
        latitude.append(lat)
        longitude.append(lng)

In [20]:
# convert to dataframe
df_nursing = pd.DataFrame(zip(names, latitude, longitude), columns = ["name", "latitude", "longitude"])
df_nursing.head()

,name,latitude,longitude
0,ALL SAINTS HOME (HOUGANG),1.365532,103.883965
1,ALL SAINTS HOME (JURONG EAST),1.341734,103.740725
2,ALL SAINTS HOME (TAMPINES),1.361600,103.954070
3,ALL SAINTS HOME (YISHUN),1.435380,103.830032
4,ALLIUM CARE SUITES,1.360119,103.828251


Concatenate the remaining nursing home list from csv file.   
Note I only mange to scrape the first page, as the url remain the same after click the next page.     
Thus I extracted the remaining list of nursing home manually and store in csv file as I am still not succesfully scrape via python

In [22]:
df_n2 = pd.read_csv('../data/sg_nursinghome.csv')
df_nursing = pd.concat([df_nursing, df_n2], ignore_index = True)
del df_n2

In [23]:
print(f"There are total of {df_nursing.shape[0]} nursing home in the list")
# export the full list of nursing home to csv
df_nursing.to_csv('../data/output/sg_nursinghome_full.csv', index = False)

There are total of 75 nursing home in the list


## 2.2 Geocoding
Based on the planning areas in Singapore datasset, I will need to convert it into a location in terms of latitude and longitude, in order to plot them into a map.
The location data (latitude, longitude) is also needed to explore the nearby location in each planning areas.

In [24]:
# placeholder to store the latitude and longitude of the planning areas of Singapore
lat, lng = [], []

geolocator = Nominatim(user_agent = 'sg_explorer')
# create for loop to get the location data for the planning areas
for name in df_sg['Name']:
    address = name + ', Singapore'
    location = geolocator.geocode(address)
    lat.append(location.latitude)
    lng.append(location.longitude)


In [25]:
# add lat, lng into df_sg
df_sg['latitude'] = lat
df_sg['longitude'] = lng
df_sg.head()

,Name,Region,Area(km2),Population,Density(/km2),latitude,longitude
0,Ang Mo Kio,North-East,13.94,163950.0,13400.0,1.370080,103.849523
1,Bedok,East,21.69,279380.0,13000.0,1.323976,103.930216
2,Bishan,Central,7.62,88010.0,12000.0,1.350986,103.848255
3,Boon Lay,West,8.23,30.0,3.6,1.338550,103.705812
4,Bukit Batok,West,11.13,153740.0,14000.0,1.349057,103.749591


There are 6 areas with inaccurate location. I manually search for the location and updated in a list. Code cell below update the dataframe with more accurate location (i.e. latitude & longitude) for these 6 areas.

In [26]:
name_list = ['Boon Lay', 'Bukit Panjang', 'Bukit Timah', 'Orchard', 'Paya Lebar', 'Pioneer']
latitude_list = [1.313735, 1.380833, 1.329411, 1.30512, 1.350399, 1.317788]
longitude_list = [103.70677, 103.7625, 103.802078, 103.832131, 103.901299, 103.664135]

In [27]:
# Update the location data
for name, lat, lng in zip(name_list, latitude_list, longitude_list):
    df_sg.loc[df_sg['Name'] == name, 'latitude'] = lat
    df_sg.loc[df_sg['Name'] == name, 'longitude'] = lng

In [28]:
# export df_sg to csv file
df_sg.to_csv('../data/output/df_sg.csv', index = False)

## 2.3 Foursquare API to explore nearby places for each area    
With the location data for each planning area, I am able to find the top 100 places for each Singapore planning area using Foursquare API.   
These data are needed to cluster the Singapore planning areas into 8 clusters (that having similarity in terms of nearby places).

Define Foursquare Credentials and Version

In [90]:
CLIENT_ID = 'JWDXHM0IORPKPQWHUFSAN0NS3F5IU1RUD42IMK43IX2FX1FD' # your Foursquare ID
CLIENT_SECRET = 'B1OP2PZ2GPDUMRS0I0UOZRLQELLKER4I5HHHARJ4MSFAR1C3' # your Foursquare Secret
VERSION = '20180622' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JWDXHM0IORPKPQWHUFSAN0NS3F5IU1RUD42IMK43IX2FX1FD
CLIENT_SECRET:B1OP2PZ2GPDUMRS0I0UOZRLQELLKER4I5HHHARJ4MSFAR1C3


In [91]:
# Function to extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Function to extract venues on each area in the df

In [92]:
def getNearbyVenues(names, latitudes, longitudes, radius = 1000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API requests url
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, CLIENT_SECRET, VERSION,
                lat, lng, radius, LIMIT)
        
        # make the GET requests
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venues
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name',
                            'Area Latitude',
                            'Area Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Categories']
    return (nearby_venues)
          
        

In [93]:
LIMIT = 100
sg_venues = getNearbyVenues(names = df_sg.Name, 
                            latitudes = df_sg.latitude, longitudes = df_sg.longitude)

ANG MO KIO
BEDOK
BISHAN
BOON LAY
BUKIT BATOK
BUKIT MERAH
BUKIT PANJANG
BUKIT TIMAH
CENTRAL WATER CATCHMENT
CHANGI
CHANGI BAY
CHOA CHU KANG
CLEMENTI
DOWNTOWN CORE
GEYLANG
HOUGANG
JURONG EAST
JURONG WEST
KALLANG
LIM CHU KANG
MANDAI
MARINA EAST
MARINA SOUTH
MARINE PARADE
MUSEUM
NEWTON
NORTH-EASTERN ISLANDS
NOVENA
ORCHARD
OUTRAM
PASIR RIS
PAYA LEBAR
PIONEER
PUNGGOL
QUEENSTOWN
RIVER VALLEY
ROCHOR
SELETAR
SEMBAWANG
SENGKANG
SERANGOON
SIMPANG
SINGAPORE RIVER
SOUTHERN ISLANDS
STRAITS VIEW
SUNGEI KADUT
TAMPINES
TANGLIN
TENGAH
TOA PAYOH
TUAS
WESTERN ISLANDS
WESTERN WATER CATCHMENT
WOODLANDS
YISHUN


In [94]:
print(f"Total list of nearby venues: {sg_venues.shape[0]}")
sg_venues.head()

Total list of nearby venues: 3130


,Name,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Categories
0,ANG MO KIO,1.370073,103.849516,Old Chang Kee,1.369094,103.848389,Snack Place
1,ANG MO KIO,1.370073,103.849516,FairPrice Xtra,1.369279,103.848886,Supermarket
2,ANG MO KIO,1.370073,103.849516,MOS Burger,1.369170,103.847831,Burger Joint
3,ANG MO KIO,1.370073,103.849516,NTUC FairPrice,1.371507,103.847082,Supermarket
4,ANG MO KIO,1.370073,103.849516,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House


Number of venues returned for each area

In [95]:
sg_venues.groupby('Name').size()

Name
ANG MO KIO                  87
BEDOK                      100
BISHAN                      72
BOON LAY                    30
BUKIT BATOK                 44
BUKIT MERAH                 55
BUKIT PANJANG               51
BUKIT TIMAH                 60
CENTRAL WATER CATCHMENT      4
CHANGI                      30
CHANGI BAY                   9
CHOA CHU KANG               55
CLEMENTI                    88
DOWNTOWN CORE              100
GEYLANG                    100
HOUGANG                     66
JURONG EAST                 88
JURONG WEST                 73
KALLANG                     86
LIM CHU KANG                 6
MANDAI                       5
MARINA EAST                 62
MARINA SOUTH                90
MARINE PARADE              100
MUSEUM                     100
NEWTON                     100
NOVENA                     100
ORCHARD                    100
OUTRAM                     100
PASIR RIS                   38
PAYA LEBAR                  10
PIONEER                      4
PUN

In [96]:
print("There are total of {} uniques categories".format(sg_venues['Venue Categories'].nunique()))

There are total of 290 uniques categories


The 15 top most frequent Venue Categories

In [97]:
sg_venues.groupby('Venue Categories')['Venue'].count().sort_values(ascending = False)[:15]

Venue Categories
Coffee Shop             170
Chinese Restaurant      152
Food Court              150
Café                    129
Hotel                    98
Japanese Restaurant      92
Asian Restaurant         87
Fast Food Restaurant     75
Bakery                   65
Supermarket              65
Shopping Mall            60
Seafood Restaurant       51
Indian Restaurant        49
Noodle House             48
Thai Restaurant          45
Name: Venue, dtype: int64

In [98]:
# export sg_venues to csv
sg_venues.to_csv('../data/output/sg_venues.csv', index = False)

## 2.4 Demographics dataset downloaded from government website

Population by geographic distribution dataset was downloaded from [Department of Statitistics Singapore's website](https://www.singstat.gov.sg/find-data/search-by-theme/population/geographic-distribution/latest-data).     
The data is used to analyze the demographics pattern of Singapore, for example which are the planning area has highest population, population density in total or based on different age group.    
This information will also be used to analyse any relationship between the distribution of nursing home and demographics trait (age, geographic location). In addition, it will be used to recommend strategic location for new nursing home if found area with low nursing home facility coverage.

### Demographics data compilation for each planning area   
There are total of 55 planning area in Singapore
The original csv file's size is exceeded the limit in github, thus do not run below code cell.   
Instead, I extract only the year 2019 dataset and uploaded to github.

In [5]:
# read csv files
#df_sg_demo = pd.read_csv('../data/Singapore Residents by Planning AreaSubzone Age Group Sex and Type of Dwelling June 20112019/respopagesextod2011to2019.csv')

# The dataset compiled the population from year 2011 to 2019.
#print(f"The dataset comprises of {df_sg_demo.Time.nunique()} years data (2011-2019)")
#print(f"for all {df_sg_demo.PA.nunique()} planning areas and")
#print(f"each area divided into {df_sg_demo.AG.nunique()} age groups (AG)")

#df_sg_demo.head(10)

The dataset comprises of 9 years data (2011-2019)
for all 55 planning areas and
each area divided into 19 age groups (AG)


,PA,SZ,AG,Sex,TOD,Pop,Time
0,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,HDB 1- and 2-Room Flats,0,2011
1,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,HDB 3-Room Flats,10,2011
2,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,HDB 4-Room Flats,30,2011
3,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,HDB 5-Room and Executive Flats,50,2011
4,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,HUDC Flats (excluding those privatised),0,2011
5,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,Landed Properties,0,2011
6,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,Condominiums and Other Apartments,40,2011
7,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,Others,0,2011
8,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Females,HDB 1- and 2-Room Flats,0,2011
9,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Females,HDB 3-Room Flats,10,2011


In [6]:
df_sg_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883728 entries, 0 to 883727
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   PA      883728 non-null  object
 1   SZ      883728 non-null  object
 2   AG      883728 non-null  object
 3   Sex     883728 non-null  object
 4   TOD     883728 non-null  object
 5   Pop     883728 non-null  int64 
 6   Time    883728 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 47.2+ MB


In [8]:
# Extract demographics for year 2019 only
#sg_2019demographics = df_sg_demo[df_sg_demo['Time'] == 2019]
# export to csv file
#sg_2019demographics.to_csv('../data/Singapore Residents by Planning AreaSubzone Age Group Sex and Type of Dwelling June 20112019/respopagesextod2019.csv',
                          index = False)

In [9]:
# Extract demographics for year 2019
df_sg_demo = pd.read_csv('../data/Singapore Residents by Planning AreaSubzone Age Group Sex and Type of Dwelling June 20112019/respopagesextod2019.csv')
# extract only the required columns
df_sg_demo.drop(['TOD','SZ','Time'], axis = 1, inplace = True)
df_sg_demo.head(3)

,PA,AG,Sex,Pop
0,Ang Mo Kio,0_to_4,Males,0
1,Ang Mo Kio,0_to_4,Males,10
2,Ang Mo Kio,0_to_4,Males,10


In [10]:
# Function to aggregate the total residents age-group (AG)
def sum_goupbylist(df):
    return list(df.groupby('AG')['Pop'].sum())

In [11]:
df_sg_demo.groupby('AG').size().index.values

array(['0_to_4', '10_to_14', '15_to_19', '20_to_24', '25_to_29',
       '30_to_34', '35_to_39', '40_to_44', '45_to_49', '50_to_54',
       '55_to_59', '5_to_9', '60_to_64', '65_to_69', '70_to_74',
       '75_to_79', '80_to_84', '85_to_89', '90_and_over'], dtype=object)

In [12]:
# Create new dataframe for total population per Planning Area (PA) per age group
demo_list = {}   # empty dict to store the total residents per age-group, key = PA, values = list of total residents per AG

for col in df_sg_demo.PA.unique():
    demo_list[col] = sum_goupbylist(df_sg_demo[df_sg_demo['PA'] == col])

# new dataframe, index = PA, columns = AG
df_sg_demo_sorted = pd.DataFrame(data = demo_list).T
df_sg_demo_sorted.columns = df_sg_demo.groupby('AG').size().index.values

df_sg_demo_sorted.head()

,0_to_4,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,5_to_9,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_to_89,90_and_over
Ang Mo Kio,5420,7380,7930,8920,10620,10510,10940,11760,12570,12170,13090,6230,12810,11970,8960,6160,3840,2110,1040
Bedok,10020,13300,14640,16660,19530,17940,18310,20070,21290,20870,22550,11640,21830,18810,13660,8300,5600,3130,1820
Bishan,2850,4430,4740,5570,7090,5430,5290,5940,6860,6510,7220,3850,7140,5730,3880,2540,1670,970,520
Boon Lay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bukit Batok,7130,7800,8800,9850,12510,12480,10600,10690,11680,12010,12450,6640,11590,8560,5020,2930,1820,1020,560


In [13]:
# rearange the column
ag_5_9 = df_sg_demo_sorted['5_to_9']

df_sg_demo_sorted.drop(columns = ['5_to_9'], axis = 1, inplace = True)
df_sg_demo_sorted.insert(1, '5_to_9', ag_5_9.values)
df_sg_demo_sorted.head()

,0_to_4,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_to_89,90_and_over
Ang Mo Kio,5420,6230,7380,7930,8920,10620,10510,10940,11760,12570,12170,13090,12810,11970,8960,6160,3840,2110,1040
Bedok,10020,11640,13300,14640,16660,19530,17940,18310,20070,21290,20870,22550,21830,18810,13660,8300,5600,3130,1820
Bishan,2850,3850,4430,4740,5570,7090,5430,5290,5940,6860,6510,7220,7140,5730,3880,2540,1670,970,520
Boon Lay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bukit Batok,7130,6640,7800,8800,9850,12510,12480,10600,10690,11680,12010,12450,11590,8560,5020,2930,1820,1020,560


Re-group to have 3 instead of 19 AG, where:
- 0-24 : Young, infants up to mostly undergrades
- 25-64 : Medium, mostly working adult
- 65 and above : Senior, mostly retiree (retirement age in Singapore is 62 years old)

In [14]:
df_sg_demo_sorted['0_to_24'] = df_sg_demo_sorted.iloc[:,:5].sum(axis=1)
df_sg_demo_sorted['25_to_64'] = df_sg_demo_sorted.iloc[:,5:-7].sum(axis=1) # added ['0_to)24'] column
df_sg_demo_sorted['65_and_over'] = df_sg_demo_sorted.iloc[:,-8:-2].sum(axis=1)
df_sg_demo_sorted.head()

,0_to_4,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,...,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_to_89,90_and_over,0_to_24,25_to_64,65_and_over
Ang Mo Kio,5420,6230,7380,7930,8920,10620,10510,10940,11760,12570,...,12810,11970,8960,6160,3840,2110,1040,35880,94470,34080
Bedok,10020,11640,13300,14640,16660,19530,17940,18310,20070,21290,...,21830,18810,13660,8300,5600,3130,1820,66260,162390,51320
Bishan,2850,3850,4430,4740,5570,7090,5430,5290,5940,6860,...,7140,5730,3880,2540,1670,970,520,21440,51480,15310
Boon Lay,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bukit Batok,7130,6640,7800,8800,9850,12510,12480,10600,10690,11680,...,11590,8560,5020,2930,1820,1020,560,40220,94010,19910


In [15]:
# reset index in order to get first column as area name
df_sg_demo_sorted.reset_index(inplace = True)

df_sg_demo_sorted.rename(columns = {'index': 'Name'}, inplace = True)
df_sg_demo_sorted['Name'] = df_sg_demo_sorted['Name'].apply(lambda x: x.upper())

In [39]:
# export df_sg_demo_sorted to csv
df_sg_demo_sorted.to_csv('../data/output/df_sg_demo_sorted.csv', index = False)

### Repeat the same to compile the demographcis for each subzone of each planning area    
There are total of 323 subzones in Singapore

In [16]:
# Extract demographics for year 2019

df_subzone_demo = pd.read_csv('../data/Singapore Residents by Planning AreaSubzone Age Group Sex and Type of Dwelling June 20112019/respopagesextod2019.csv')
df_subzone_demo.drop(['TOD', 'Time'], axis=1, inplace = True)
df_subzone_demo.head(3)

,PA,SZ,AG,Sex,Pop
0,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,0
1,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,10
2,Ang Mo Kio,Ang Mo Kio Town Centre,0_to_4,Males,10


In [17]:
demo_subzone = {}
for col in df_subzone_demo.SZ.unique():
    demo_subzone[col] = sum_goupbylist(df_subzone_demo[df_subzone_demo['SZ'] == col])

# new dataframe, index = SZ, columns = AG
df_subzone_demo_sorted = pd.DataFrame(data = demo_subzone).T
df_subzone_demo_sorted.columns = df_sg_demo.groupby('AG').size().index.values

df_subzone_demo_sorted.head()

,0_to_4,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,5_to_9,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_to_89,90_and_over
Ang Mo Kio Town Centre,170,330,310,290,290,280,330,430,470,350,350,260,290,270,200,140,80,40,10
Cheng San,1050,1060,1210,1380,1810,2010,2220,2050,2220,2070,2160,1000,2200,2150,1570,1020,580,310,150
Chong Boon,840,1020,1150,1380,1600,1930,1820,1900,1840,1980,2150,880,2160,2080,1670,1190,740,400,200
Kebun Bahru,730,1020,1090,1170,1450,1400,1550,1700,1860,1820,1800,840,1750,1700,1300,900,590,330,140
Sembawang Hills,200,410,460,530,510,320,290,400,540,540,550,280,480,420,300,240,150,100,50


In [18]:
# rearange the column
ag_5_9 = df_subzone_demo_sorted['5_to_9']

df_subzone_demo_sorted.drop(columns = ['5_to_9'], axis = 1, inplace = True)
df_subzone_demo_sorted.insert(1, '5_to_9', ag_5_9.values)
df_subzone_demo_sorted.head()

,0_to_4,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_to_89,90_and_over
Ang Mo Kio Town Centre,170,260,330,310,290,290,280,330,430,470,350,350,290,270,200,140,80,40,10
Cheng San,1050,1000,1060,1210,1380,1810,2010,2220,2050,2220,2070,2160,2200,2150,1570,1020,580,310,150
Chong Boon,840,880,1020,1150,1380,1600,1930,1820,1900,1840,1980,2150,2160,2080,1670,1190,740,400,200
Kebun Bahru,730,840,1020,1090,1170,1450,1400,1550,1700,1860,1820,1800,1750,1700,1300,900,590,330,140
Sembawang Hills,200,280,410,460,530,510,320,290,400,540,540,550,480,420,300,240,150,100,50


In [19]:
# Added 3 age-group: Young, Medium, Senior age group
df_subzone_demo_sorted['0_to_24'] = df_subzone_demo_sorted.iloc[:,:5].sum(axis=1)
df_subzone_demo_sorted['25_to_64'] = df_subzone_demo_sorted.iloc[:,5:-7].sum(axis=1) # added ['0_to)24'] column
df_subzone_demo_sorted['65_and_over'] = df_subzone_demo_sorted.iloc[:,-8:-2].sum(axis=1)
df_subzone_demo_sorted['total_population'] = df_subzone_demo_sorted.iloc[:,:-3].sum(axis=1)
df_subzone_demo_sorted.head()

,0_to_4,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,...,65_to_69,70_to_74,75_to_79,80_to_84,85_to_89,90_and_over,0_to_24,25_to_64,65_and_over,total_population
Ang Mo Kio Town Centre,170,260,330,310,290,290,280,330,430,470,...,270,200,140,80,40,10,1360,2790,740,4890
Cheng San,1050,1000,1060,1210,1380,1810,2010,2220,2050,2220,...,2150,1570,1020,580,310,150,5700,16740,5780,28220
Chong Boon,840,880,1020,1150,1380,1600,1930,1820,1900,1840,...,2080,1670,1190,740,400,200,5270,15380,6280,26930
Kebun Bahru,730,840,1020,1090,1170,1450,1400,1550,1700,1860,...,1700,1300,900,590,330,140,4850,13330,4960,23140
Sembawang Hills,200,280,410,460,530,510,320,290,400,540,...,420,300,240,150,100,50,1880,3630,1260,6770


In [20]:
# reset index in order to get first column as area name
df_subzone_demo_sorted.reset_index(inplace = True)

df_subzone_demo_sorted.rename(columns = {'index': 'Subzone'}, inplace = True)
# change subzone to upper letter (to match with geojson/ shp file subzone naming)
df_subzone_demo_sorted['Subzone'] = df_subzone_demo_sorted['Subzone'].apply(lambda x: x.upper())

df_subzone_demo_sorted.shape

(323, 24)

In [48]:
# export to csv
df_subzone_demo_sorted.to_csv('../data/output/df_subzone_demo_sorted.csv', index = False)

In [49]:
subzone_list = list(df_subzone_demo_sorted['Subzone'])

In [50]:
df_subzone_demo_sorted.head()

,Subzone,0_to_4,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,...,65_to_69,70_to_74,75_to_79,80_to_84,85_to_89,90_and_over,0_to_24,25_to_64,65_and_over,total_population
0,ANG MO KIO TOWN CENTRE,170,260,330,310,290,290,280,330,430,...,270,200,140,80,40,10,1360,2790,740,4890
1,CHENG SAN,1050,1000,1060,1210,1380,1810,2010,2220,2050,...,2150,1570,1020,580,310,150,5700,16740,5780,28220
2,CHONG BOON,840,880,1020,1150,1380,1600,1930,1820,1900,...,2080,1670,1190,740,400,200,5270,15380,6280,26930
3,KEBUN BAHRU,730,840,1020,1090,1170,1450,1400,1550,1700,...,1700,1300,900,590,330,140,4850,13330,4960,23140
4,SEMBAWANG HILLS,200,280,410,460,530,510,320,290,400,...,420,300,240,150,100,50,1880,3630,1260,6770


## 2.5 Singapore planning areas' boundary dataset

The polygon of planning areas boundary as well as subzone boundary can be downloaded from government website [Data.gov.sg](https://data.gov.sg/dataset/master-plan-2014-planning-area-boundary-web). The dataset is in `kml` format and it was then converted to `geojson` format.
This is the version as published at URA website and OneMap.   
They are needed to plot the **choropleth map** of singapore to visualise the demographics pattern of Singapore.

In [51]:
# Load geojson file
with open('../data/master-plan-2014-planning-area-boundary-web/MP14_PLNG_AREA_WEB_PL.geojson') as f:
    data = json.load(f)
    
data['features'][0]['properties']

{'Name': 'BUKIT MERAH',
 'description': None,
 'X_ADDR': '26865.7838',
 'SHAPE_Length': '29156.293788',
 'Central Area Indicator': 'N',
 'Region Name': 'CENTRAL REGION',
 'Region Code': 'CR',
 'Planning Area Name': 'BUKIT MERAH',
 'FMEL_UPD_D': '12/5/2014 9:22:21 PM',
 'SHAPE': 'Polygon',
 'Y_ADDR': '28662.8732',
 'INC_CRC': '738B479882E4EE28',
 'SHAPE_Area': '14462471.888142',
 'Planning Area Code': 'BM',
 'Field_1': 'BUKIT MERAH'}

In [52]:
# parse geojson data
# data to extract is the Name from data['features']['properties']

name = []
#region = []
for feature in data['features']:
    name.append(feature['properties']['Name'])
    #region.append(feature['properties']['Region Name'])

# convert to df
sg_geo_df = pd.DataFrame(data = name, columns = ['Name'])
sg_geo_df.head()

,Name
0,BUKIT MERAH
1,CHOA CHU KANG
2,BUKIT TIMAH
3,CENTRAL WATER CATCHMENT
4,CHANGI


Create new datataframe `df_sg_geo` that consists of population density data per planning area, but with the same order as in the geojson file that has the polygon data of each planning area.

In [53]:
df_sg['Name'] = df_sg['Name'].apply(lambda x:x.upper())
# merge the df_sg (that includes the population density per planning area) with sg_geo_df
df_sg_pop = sg_geo_df.merge(df_sg, how = 'inner', on = 'Name')
df_sg_pop.head()

,Name,Region,Area(km2),Population,Density(/km2),latitude,longitude
0,BUKIT MERAH,Central,14.34,151980.0,11000.00,1.270439,103.828318
1,CHOA CHU KANG,West,6.11,190890.0,30000.00,1.384749,103.744534
2,BUKIT TIMAH,Central,17.53,77430.0,4400.00,1.329411,103.802078
3,CENTRAL WATER CATCHMENT,North,37.15,0.0,0.00,1.375708,103.801743
4,CHANGI,East,40.61,1830.0,80.62,1.351080,103.990064


Combine the df_sg_geo with the demographics data (that divided into 3 AG)

In [54]:
# merge df_sg_demo_sorted to sg_geo_pop_df.head()
df_sg_geo_pop = df_sg_pop.merge(df_sg_demo_sorted[['Name','0_to_24','25_to_64','65_and_over']],
                                    how = 'inner', on = 'Name')
df_sg_geo_pop.head()

,Name,Region,Area(km2),Population,Density(/km2),latitude,longitude,0_to_24,25_to_64,65_and_over
0,BUKIT MERAH,Central,14.34,151980.0,11000.00,1.270439,103.828318,32620,87510,32470
1,CHOA CHU KANG,West,6.11,190890.0,30000.00,1.384749,103.744534,57290,114980,18830
2,BUKIT TIMAH,Central,17.53,77430.0,4400.00,1.329411,103.802078,22480,42530,12710
3,CENTRAL WATER CATCHMENT,North,37.15,0.0,0.00,1.375708,103.801743,0,0,0
4,CHANGI,East,40.61,1830.0,80.62,1.351080,103.990064,590,1070,130


In [55]:
#export df_sg_geo_pop to csv
df_sg_geo_pop.to_csv('../data/output/df_sg_geo_pop.csv', index = False)

The polygon of **subzone** for each planning areas, downloaded the `.shp` format and read it as `GeoDataFrame`.

In [56]:
subzone = gpd.read_file('../data/master-plan-2014-subzone-boundary-no-sea/master-plan-2014-subzone-boundary-no-sea-shp/MP14_SUBZONE_NO_SEA_PL.shp')
# extract the desired column only
subzone = subzone[['SUBZONE_N', 'PLN_AREA_N', 'X_ADDR', 'Y_ADDR', 'geometry']]
subzone.head()

,SUBZONE_N,PLN_AREA_N,X_ADDR,Y_ADDR,geometry
0,PEOPLE'S PARK,OUTRAM,28831.7807,29419.6457,"POLYGON ((29099.021 29640.030, 29116.963 29625..."
1,BUKIT MERAH,BUKIT MERAH,26360.7990,29384.1429,"POLYGON ((26750.092 29216.098, 26751.912 29173..."
2,CHINATOWN,OUTRAM,29153.9676,29158.0443,"POLYGON ((29161.201 29723.071, 29189.033 29704..."
3,PHILLIP,DOWNTOWN CORE,29706.7242,29744.9079,"POLYGON ((29814.107 29616.894, 29806.682 29593..."
4,RAFFLES PLACE,DOWNTOWN CORE,29968.6175,29572.7618,"POLYGON ((30137.768 29843.194, 30138.417 29841..."


In [58]:
# Get the CRS (Coordinate Reference System) of the datset.
# EPSG:3414 SVY21 / Singapore TM
subzone.crs

<Projected CRS: PROJCS["SVY21",GEOGCS["SVY21[WGS84]",DATUM["WGS_19 ...>
Name: SVY21
Axis Info [cartesian]:
- E[east]: Easting (Meter)
- N[north]: Northing (Meter)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS_84
- Prime Meridian: Greenwich

In [59]:
# Convert the geometry to epsg4326 (WGS84 Latitude/Longitude)
subzone_epsg4326 = subzone.to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
subzone_epsg4326.head()

,SUBZONE_N,PLN_AREA_N,X_ADDR,Y_ADDR,geometry
0,PEOPLE'S PARK,OUTRAM,28831.7807,29419.6457,"POLYGON ((103.84319 1.28433, 103.84335 1.28420..."
1,BUKIT MERAH,BUKIT MERAH,26360.7990,29384.1429,"POLYGON ((103.82209 1.28049, 103.82210 1.28011..."
2,CHINATOWN,OUTRAM,29153.9676,29158.0443,"POLYGON ((103.84375 1.28508, 103.84400 1.28491..."
3,PHILLIP,DOWNTOWN CORE,29706.7242,29744.9079,"POLYGON ((103.84962 1.28412, 103.84955 1.28391..."
4,RAFFLES PLACE,DOWNTOWN CORE,29968.6175,29572.7618,"POLYGON ((103.85253 1.28617, 103.85253 1.28615..."


In [60]:
# check the 'path' that store the subzone, it is under: features.properties.SUBZONE_N
subzone_epsg4326.__geo_interface__

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'PLN_AREA_N': 'OUTRAM',
    'SUBZONE_N': "PEOPLE'S PARK",
    'X_ADDR': 28831.7807,
    'Y_ADDR': 29419.6457},
   'geometry': {'type': 'Polygon',
    'coordinates': (((103.84319371624179, 1.2843283894118063),
      (103.84335493257842, 1.284195446235938),
      (103.84330679758429, 1.284134566388901),
      (103.84312109644638, 1.2838996935977303),
      (103.84292296042989, 1.283657388768436),
      (103.84271157368734, 1.2834133837658397),
      (103.84261017759268, 1.2832908743454594),
      (103.8424571404303, 1.2831009763552979),
      (103.84227963510702, 1.2828915034631463),
      (103.84212615689785, 1.2827272785649144),
      (103.84194728225297, 1.282517488221228),
      (103.84181425845159, 1.282361427840525),
      (103.84170715132352, 1.2822373203887407),
      (103.84155214022545, 1.2820551121631074),
      (103.84134100262497, 1.2818063744992512),
      (103.84111557914026, 1.